<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/ml_Solutions/OpenAI_vs_Google_vs_Anthropic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multimodal Models

# OpenAI
### OpenAi Pricing
https://openai.com/api/pricing/

###Gpt-4o

https://openai.com/index/hello-gpt-4o/

#####Context Window
https://community.openai.com/t/announcing-gpt-4o-in-the-api/744700

GPT-4o has a 128K context window and has a knowledge cut-off date of October 2023.


# Anthropic
https://www.anthropic.com/news/claude-3-family

###Claude 3
https://docs.anthropic.com/en/docs/models-overview

#####Context Window
https://www.anthropic.com/news/claude-3-family

The Claude 3 family of models will initially offer a 200K context window upon launch. However, all three models are capable of accepting inputs exceeding 1 million tokens and we may make this available to select customers who need enhanced processing power.

### pricing
https://www.anthropic.com/api#pricing

# Google Gemini

### Gemini 1.5 flask

https://deepmind.google/technologies/gemini/flash/



### pricing
https://ai.google.dev/pricing

https://github.com/anthropics/anthropic-sdk-python

#####Context Window
https://deepmind.google/technologies/gemini/flash/

Flash has a one-million-token context window by default, which means you can process one hour of video, 11 hours of audio, codebases with more than 30,000 lines of code, or over 700,000 words.

In [9]:
!pip install tiktoken openai voyageai anthropic google-cloud-aiplatform  google-generativeai  python-dotenv -q

In [13]:
!curl ipinfo.io

{
  "ip": "104.199.230.178",
  "hostname": "178.230.199.104.bc.googleusercontent.com",
  "city": "Taipei",
  "region": "Taiwan",
  "country": "TW",
  "loc": "25.0478,121.5319",
  "org": "AS396982 Google LLC",
  "timezone": "Asia/Taipei",
  "readme": "https://ipinfo.io/missingauth"
}

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import tiktoken
from openai import OpenAI
from dotenv import dotenv_values
import json
import vertexai
from google.oauth2 import service_account
import base64
from vertexai.generative_models import GenerativeModel, Part, ChatSession
import vertexai.preview.generative_models as generative_models

In [7]:
from google.colab import userdata
# OpenAI and ANTROPHIC credentials
api_key = userdata.get('KEY_OPENAI')
API_KEY_ANTROPIC = userdata.get("ANTROPIC_KEY")

In [11]:
config = dotenv_values("/content/drive/MyDrive/keys/keys/.env")
with open("/content/drive/MyDrive/keys/keys/complete-tube-421007-9a7c35cd44e2.json") as source:
    info = json.load(source)

#Gemini Pro flash

In [15]:
#vertex AI credentials
vertex_credentials = service_account.Credentials.from_service_account_info(info)

vertexai.init(
        project=config["PROJECT"],
        location=config["REGION"],
        credentials=vertex_credentials,
    )

In [69]:
def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response), responses

In [50]:
model = GenerativeModel(
    "gemini-1.5-flash-preview-0514",
    system_instruction=["""You a helpful agent who helps to extract relevant information from documents"""],
    safety_settings= {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_NONE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_NONE,
},
     generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0.1,
    "top_p": 0.95,
}
  )
chat = model.start_chat(response_validation=False)

In [64]:
from pathlib import Path
content=[]
base64_images = []
pathlist = Path("/content/drive/MyDrive/data (1)/docs/").glob('**/*.jpg')
for path in pathlist:
    # Convert path object to a string
    path_in_str = str(path)
    # Perform actions on the file (e.g., print its path)
    print(path_in_str)
    with open(path_in_str, "rb") as f:
        im_bytes = f.read()
        im_b64 = base64.b64encode(im_bytes).decode("utf8")
        base64_images.append(im_b64)
        image = Part.from_data(data=im_b64, mime_type="image/jpeg")
        content.append(image)

/content/drive/MyDrive/data (1)/docs/Halifax - Bank Statement 1.jpg


In [60]:
base64_images[0][:100]

'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAx'

In [65]:
question = """Intructions:
              1) Extract bank name
              2) Date of statement
              3) Dont show names or surnames. refers as subject Mr X or Mrs X.
              4) Money In, Money Out and closing Balance
              5) If the Language is not English , respond in English together with the document language

"""

In [80]:
prompt =[f"""{question} """] +  content

In [81]:
model.count_tokens(prompt)

total_tokens: 330
total_billable_characters: 207

In [70]:
response, res = get_chat_response(chat, prompt)

In [71]:
print(response)

The bank name is Halifax.

The date of the statement is 11 November 2017.

Mr/Mrs X's Money In is 9,070.00.

Mr/Mrs X's Money Out is 9,741.17.

Mr/Mrs X's closing balance is 3,815.97. 



In [78]:
chat.history[-1]

role: "model"
parts {
  text: "The bank name is Halifax.\n\nThe date of the statement is 11 November 2017.\n\nMr/Mrs X\'s Money In is 9,070.00.\n\nMr/Mrs X\'s Money Out is 9,741.17.\n\nMr/Mrs X\'s closing balance is 3,815.97. \n"
}

In [79]:
model.count_tokens(prompt)

total_tokens: 330
total_billable_characters: 207

In [82]:

client = OpenAI(api_key=api_key)


response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "you are a document extractor tool"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": question
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_images[0]}"
          }
        }

      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": "",
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [83]:
response.usage

CompletionUsage(completion_tokens=79, prompt_tokens=1200, total_tokens=1279)

In [86]:
print(response.choices[0].message.content)

1) **Bank Name**: Halifax

2) **Date of Statement**: 11 November 2017

3) **Money In, Money Out, and Closing Balance**:
   - **Money In**: £9,070.00
   - **Money Out**: £9,741.17
   - **Closing Balance**: £3,815.97


# Claude 3


In [88]:
import os
from anthropic import Anthropic

In [94]:
# ANTHROPIC_API_KEY
os.environ["ANTHROPIC_API_KEY"] = API_KEY_ANTROPIC

In [95]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=API_KEY_ANTROPIC,
)

In [108]:
# claude-3-haiku-20240307 # claude-3-opus-20240229
message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=2048,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": f"{base64_images[0]}",
                    },
                },
                {
                    "type": "text",
                    "text": question
                }
            ],
        }
    ],
)
print(message)

Message(id='msg_01HaWKA5ShYy2Z58XGNTVJai', content=[TextBlock(text='1) The bank name is Halifax.\n\n2) The date of the statement is 11 November 2017.\n\n3) The statement does not contain any names or surnames that can be identified. The transactions are described with the payment type or vendor name.\n\n4) The Money In column shows a single credit of £5.00 on 04 Oct 17. The Money Out column shows various debit transactions totaling £3,750.83. The closing Balance is £3,815.97.\n\n5) The statement is in English.', type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=1634, output_tokens=121))


In [109]:
print(message.content[0].text)

1) The bank name is Halifax.

2) The date of the statement is 11 November 2017.

3) The statement does not contain any names or surnames that can be identified. The transactions are described with the payment type or vendor name.

4) The Money In column shows a single credit of £5.00 on 04 Oct 17. The Money Out column shows various debit transactions totaling £3,750.83. The closing Balance is £3,815.97.

5) The statement is in English.


In [100]:
print(message.usage)

Usage(input_tokens=1634, output_tokens=95)


In [101]:
#cost Anthropic

message.usage.input_tokens * (15/10**6) + message.usage.output_tokens * (75/10**6)

0.031634999999999996

In [107]:

message.usage.input_tokens * (0.25/10**6) + message.usage.output_tokens * (1.25/10**6)

0.0005272499999999999

In [104]:
#cost OpenAi

response.usage.completion_tokens * (5/10**6) + response.usage.completion_tokens * (15/10**6)

0.00158

In [105]:
#cost Gemini
model.count_tokens(prompt)

total_tokens: 330
total_billable_characters: 207


In [106]:
330 * (0.70/10**6)

0.00023099999999999998